In [12]:
import sys

parent_dir = r'C:\Users\Wint3\Documents\00\ESPRIT\sem8\Deep-Learning\DL-Projet\mask-rcnn\pdl'
sys.path.append(parent_dir)

import os
import random
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from PIL import Image

# Function to resize the images and update bounding box coordinates
def resize_image(image_path, xml_path, new_image_path, new_xml_path, size=(1024, 1024)):
    # Read the image
    img = cv2.imread(image_path)
    height, width = img.shape[:2]

    # Resize the image
    img_resized = cv2.resize(img, size)

    # Parse the XML file
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Update image size in XML
    root.find('size/width').text = str(size[0])
    root.find('size/height').text = str(size[1])

    # Update bounding box coordinates
    scale_x = size[0] / width
    scale_y = size[1] / height
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = int(float(bbox.find('xmin').text) * scale_x)
        ymin = int(float(bbox.find('ymin').text) * scale_y)
        xmax = int(float(bbox.find('xmax').text) * scale_x)
        ymax = int(float(bbox.find('ymax').text) * scale_y)
        bbox.find('xmin').text = str(xmin)
        bbox.find('ymin').text = str(ymin)
        bbox.find('xmax').text = str(xmax)
        bbox.find('ymax').text = str(ymax)

    # Write the updated XML
    tree.write(new_xml_path)

    # Save the resized image
    cv2.imwrite(new_image_path, img_resized)


# Function to apply random rotation and flipping
def augment_image(image, xml_file, output_dir):
    # Generate a random angle for rotation
    angle = random.randint(0, 360)

    # Perform rotation
    rotation_matrix = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))

    # Randomly flip horizontally and vertically
    flip_code = random.randint(-1, 1)  # -1 for both, 0 for horizontal, 1 for vertical
    flipped_image = cv2.flip(rotated_image, flip_code)

    # Save augmented image
    augmented_image_path = os.path.join(output_dir, 'images', f'{os.path.basename(xml_file).split(".")[0]}_augmented.jpg')
    cv2.imwrite(augmented_image_path, flipped_image)

    # Update annotations for augmented image
    new_xml_file = os.path.join(output_dir, 'annots', f'{os.path.basename(xml_file).split(".")[0]}_augmented.xml')
    resize_image(augmented_image_path, xml_file, augmented_image_path, new_xml_file)

    return flipped_image, new_xml_file


# Function to visualize and verify transformations
def visualize_transformations(image, xml_file):
    # Read the original XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Get bounding box coordinates
    bboxes = []
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        bboxes.append([(xmin, ymin), (xmax, ymax)])

    # Plot the original image with bounding boxes
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    for bbox in bboxes:
        plt.gca().add_patch(plt.Rectangle((bbox[0][0], bbox[0][1]), bbox[1][0] - bbox[0][0], bbox[1][1] - bbox[0][1],
                                          linewidth=2, edgecolor='r', facecolor='none'))
    plt.title('Original Image with Bounding Boxes')
    plt.axis('off')
    plt.show()


# Main function to process images and annotations
def main(input_dir, output_dir):
    # Create output directories if they don't exist
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'annots'), exist_ok=True)

    # Choose a random XML file
    xml_files = [f for f in os.listdir(input_dir) if f.endswith('.xml')]
    xml_file = random.choice(xml_files)

    # Read the image corresponding to the XML file
    image_file = os.path.join(input_dir, '..', 'images', xml_file.replace('.xml', '.jpg'))
    image = cv2.imread(image_file)

    # Visualize original image and bounding boxes
    visualize_transformations(image, os.path.join(input_dir, xml_file))

    # Augment image
    augmented_image, new_xml_file = augment_image(image, os.path.join(input_dir, xml_file), output_dir)

    # Visualize augmented image and bounding boxes
    visualize_transformations(augmented_image, new_xml_file)


if __name__ == "__main__":
    # Define input and output directories
    input_dir = r'C:\Users\Wint3\Documents\00\ESPRIT\sem8\Deep-Learning\DL-Projet\mask-rcnn\pdl\WPU VHR-10 dataset\annots'
    output_dir = r'C:\Users\Wint3\Documents\00\ESPRIT\sem8\Deep-Learning\DL-Projet\mask-rcnn\pdl\WPU VHR-10 dataset augmented'

    # Call the main function
    main(input_dir, output_dir)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Wint3\\Documents\\00\\ESPRIT\\sem8\\Deep-Learning\\DL-Projet\\mask-rcnn\\pdl\\WPU VHR-10 dataset\\annots'